In [43]:
# imports
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mticks
import matplotlib.dates as mdates

In [3]:
# data
tesla_data = pd.read_csv('/Users/marcolaureano2/Documents/GitHub/tesla_ticker/data/tesla_stock_data.csv')

In [4]:
tesla_data

,Date,Year,Open,High,Low,Close,Volume,Adj Close
0,2020-01-02,2020,28.299999,28.713333,28.114000,28.684000,142981500,28.684000
1,2020-01-03,2020,29.366667,30.266666,29.128000,29.534000,266677500,29.534000
2,2020-01-06,2020,29.364668,30.104000,29.333332,30.102667,151995000,30.102667
3,2020-01-07,2020,30.760000,31.441999,30.224001,31.270666,268231500,31.270666
4,2020-01-08,2020,31.580000,33.232666,31.215334,32.809334,467164500,32.809334
...,...,...,...,...,...,...,...,...
991,2023-12-22,2023,256.760010,258.220001,251.369995,252.539993,93249800,252.539993
992,2023-12-26,2023,254.490005,257.970001,252.910004,256.609985,86892400,256.609985
993,2023-12-27,2023,258.350006,263.339996,257.519989,261.440002,106494400,261.440002
994,2023-12-28,2023,263.660004,265.130005,252.710007,253.179993,113619900,253.179993


In [5]:
tesla_data = tesla_data.set_index(['Date'])
tesla_data

,Year,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,,
2020-01-02,2020,28.299999,28.713333,28.114000,28.684000,142981500,28.684000
2020-01-03,2020,29.366667,30.266666,29.128000,29.534000,266677500,29.534000
2020-01-06,2020,29.364668,30.104000,29.333332,30.102667,151995000,30.102667
2020-01-07,2020,30.760000,31.441999,30.224001,31.270666,268231500,31.270666
2020-01-08,2020,31.580000,33.232666,31.215334,32.809334,467164500,32.809334
...,...,...,...,...,...,...,...
2023-12-22,2023,256.760010,258.220001,251.369995,252.539993,93249800,252.539993
2023-12-26,2023,254.490005,257.970001,252.910004,256.609985,86892400,256.609985
2023-12-27,2023,258.350006,263.339996,257.519989,261.440002,106494400,261.440002


In [6]:
tesla_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 996 entries, 2020-01-02 to 2023-12-29
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Year       996 non-null    int64  
 1   Open       996 non-null    float64
 2   High       996 non-null    float64
 3   Low        996 non-null    float64
 4   Close      996 non-null    float64
 5   Volume     996 non-null    int64  
 6   Adj Close  996 non-null    float64
dtypes: float64(5), int64(2)
memory usage: 62.2+ KB


In [17]:
tesla_data['Year'] = tesla_data['Year'].astype(object)

In [31]:
tesla_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 996 entries, 2020-01-02 to 2023-12-29
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Year       996 non-null    object 
 1   Open       996 non-null    float64
 2   High       996 non-null    float64
 3   Low        996 non-null    float64
 4   Close      996 non-null    float64
 5   Volume     996 non-null    int64  
 6   Adj Close  996 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 94.5+ KB


In [32]:
tesla_data['Year'].unique()

array([2020, 2021, 2022, 2023], dtype=object)

## Open

In [33]:
tesla_data['Open'].min()

24.979999542236328

In [34]:
tesla_data['Open'].max()

411.4700012207031

## High

In [35]:
tesla_data['High'].min()

26.99066734313965

In [36]:
tesla_data['High'].max()

414.4966735839844

## Low

In [37]:
tesla_data['Low'].min()

23.367332458496094

In [38]:
tesla_data['Low'].max()

405.6666564941406

## Volume

In [39]:
tesla_data['Volume'].min()

29401800

In [40]:
tesla_data['Volume'].max()

914082000

## Adj Close

In [41]:
tesla_data['Adj Close'].min()

24.08133316040039

In [42]:
tesla_data['Adj Close'].max()

409.9700012207031